In [1]:
import requests
import pandas as pd
import os

# Foursquare

#### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [18]:
bike_stations = pd.read_csv('bike_stations.csv')

url = "https://api.foursquare.com/v3/places/search"

headers = {
           'accept' : 'application/JSON',
           'Authorization' : os.getenv('FOURSQUARE_API_KEY')
          }

def request_places(row):
    params = {
             'll' : f"{row['latitude']},{row['longitude']}",
             'radius' : 1000,
             'query' : 'restaurant'
             }
    response = requests.get(url, headers=headers, params=params)
    res = response.json()
    places = pd.Series([row['station_name'], res])
    return places

foursquare_places = bike_stations.apply(request_places, axis=1)

#### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [17]:
nearest_station_name = []
restaurant_name = []
distance = []
address = []
categories = []

def get_details(row):
    for restaurant in row[1]['results']:
        restaurant_name.append(restaurant['name'])
        distance.append(restaurant['distance'])
        address.append(restaurant['location']['address'])
        nearest_station_name.append(row[0])

        restaurant_categories = []
        for category in restaurant['categories']:  # add all categories into string
            restaurant_categories.append(category['name'])
        categories.append(', '.join(restaurant_categories))

foursquare_places.apply(get_details, axis=1)

foursquare_restaurants = {
                         'restaurant_name' : restaurant_name,
                         'categories' : categories,
                         'address' : address,
                         'station_name' : nearest_station_name,
                         'distance' : distance,
                        }

#### Put your parsed results into a DataFrame

In [19]:
foursquare_restaurants = pd.DataFrame(foursquare_restaurants)

foursquare_restaurants.head()

,restaurant_name,categories,address,station_name,distance
0,Pizzeria Alle Scalette,Pizzeria,Viale Curtatone 18,Curtatone,22
1,Zest Ristorante & Winebar,"Wine Bar, Cafe, Coffee, and Tea House, Italian...",Costa di Sant'Antonio 13,Curtatone,179
2,Osteria Il Campaccio,Trattoria,Vicolo del Campaccio 2,Curtatone,176
3,Morbidi,Deli,Via B. di Sopra 73,Curtatone,248
4,Osteria L'Osteria,Italian Restaurant,Via dei Rossi 79,Curtatone,342


# Yelp

#### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [20]:
url = "https://api.yelp.com/v3/businesses/search"

headers = {'accept' : 'application/JSON',
           'Authorization' : 'Bearer ' + os.getenv('YELP_KEY')}

def request_places(row):
    params = {
              'latitude' : row['latitude'],
              'longitude' : row['longitude'],
              'radius' : 1000,
              'categories' : 'restaurants'
             }
    response = requests.get(url, headers=headers, params=params)
    res = response.json()
    places = pd.Series([row['station_name'], res])
    return places

yelp_places = bike_stations.apply(request_places, axis=1)

#### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
nearest_station_name = []
restaurant_name = []
distance = []
address = []
review_count = []
rating = []
categories = []

def get_details(row):
    for restaurant in row[1]['businesses']:
        restaurant_name.append(restaurant['name'])
        distance.append(restaurant['distance'])
        address.append(restaurant['location']['address1'])
        review_count.append(restaurant['review_count'])
        rating.append(restaurant['rating'])

        restaurant_categories = []
        for category in restaurant['categories']:  # add all categories into string
            restaurant_categories.append(category['title'])
        categories.append(', '.join(restaurant_categories))

        nearest_station_name.append(row[0])
        

yelp_places.apply(get_details, axis=1)

yelp_restaurants = {
                    'restaurant_name' : restaurant_name,
                    'categories' : categories,
                    'address' : address,
                    'review_count' : review_count,
                    'rating' : rating,
                    'station_name' : nearest_station_name,
                    'distance' : distance
                    }
                    

#### Put your parsed results into a DataFrame

In [9]:
yelp_restaurants = pd.DataFrame(yelp_restaurants)

yelp_restaurants.head()

,restaurant_name,categories,address,review_count,rating,station_name,distance
0,La Taverna di San Giuseppe,Tuscan,Via Giovanni Dupre 132,169,4.8,Curtatone,745.395007
1,Zest,"Italian, Wine Bars",Costa di Sant'antonio 13,26,4.7,Curtatone,177.848119
2,Compagnia dei Vinattieri,Tuscan,"Via delle Terme, 79",24,4.3,Curtatone,231.846606
3,La Chiacchiera,Italian,Via Costa di San Antonio 4,23,4.3,Curtatone,181.200690
4,Café Nannini,Cafes,Piazza Matteotti 32,14,4.4,Curtatone,60.451222


# Comparing Results

#### Which API provided you with more complete data? Provide an explanation. 

In [10]:
print(f'Yelp Shape: {yelp_restaurants.shape}')
print(f'Foursquare Shape: {foursquare_restaurants.shape}')

Yelp Shape: (214, 7)
Foursquare Shape: (161, 5)


The Yelp API includes more detailed data as well as has higher number of restaurants compared to the Foursquare API. With this information the Yelp API provides better coverage of data than the Foursquare API.

#### Get the top 10 restaurants according to their rating

In [11]:
yelp_restaurants.sort_values(['rating', 'review_count'], ascending=False).head(10)

,restaurant_name,categories,address,review_count,rating,station_name,distance
174,Mugolone,"Italian, Gluten-Free",Via dei Pellegrini 8,7,5.0,Casa dell'Ambiente,968.765622
198,Mugolone,"Italian, Gluten-Free",Via dei Pellegrini 8,7,5.0,Mercato,253.346444
59,L'orto de' pecci,Italian,Via di Porta Giustizia 39,2,5.0,S. Girolamo,250.275712
102,Albergo Certosa di Maggiano,"Hotels & Travel, Restaurants","Str. di Certosa, 82",2,5.0,Due Ponti,822.545866
101,Havana Club,"Cuban, Steakhouses",Piazza Maestri del Lavoro 27,1,5.0,Due Ponti,217.539983
106,Antica Trattoria Botteganova,"Italian, Bars",Strada Chiantigiana 29,1,5.0,Napoli,659.400589
144,La Grotta di San Francesco,"Seafood, Tuscan",Via Camollia 25,1,5.0,Terminal Bus,1000.171874
147,Pasta Mondo,"Pizza, Italian, Pasta Shops",Via Camollia 173,1,5.0,Terminal Bus,660.627489
148,Cartazucchero,"Used Bookstore, Bookstores, Cafeteria",Via Camollia 92,1,5.0,Terminal Bus,795.297447
152,Medo Cucina Arte & Vino,Vegetarian,Piazza del Sale 3,1,5.0,Terminal Bus,967.184559


#### Turn API DataFrame's into CSV's

In [13]:
foursquare_restaurants.to_csv('foursquare_restaurants.csv', index=False)

yelp_restaurants.to_csv('yelp_restaurants.csv', index=False)